# **Diagnóstico Automatizado de Linfomas Malignos en Biopsias H&E mediante Aprendizaje Automático**
## *Aftab Ahmed Choudhry*

In [ ]:
from PIL import Image
import os
import glob
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.applications import InceptionV3, MobileNetV2, DenseNet201, EfficientNetV2B0, ResNet50V2, VGG16 
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, RandomFlip, RandomRotation, RandomZoom, RandomTranslation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
import seaborn as sns
from itertools import cycle

## ***Conversión Única de Imágenes .tif a .png***

Esta celda se ejecuta **una sola vez** para convertir las imágenes originales descargadas desde Kaggle (en formato `.tif`) al formato `.png`, ya que `image_dataset_from_directory()` de TensorFlow **no soporta `.tif`**. Tras la conversión, las imágenes `.tif` se eliminan automáticamente.

In [ ]:
base_path = './dataset'

for class_name in ['CLL', 'FL', 'MCL']:
    folder = os.path.join(base_path, class_name)
    tif_files = glob.glob(os.path.join(folder, '*.tif'))
    png_files = glob.glob(os.path.join(folder, '*.png'))

    if len(png_files) > 0:
        print(f"Conversión ya realizada para la clase {class_name}. Se omite.")
        continue

    for tif_path in tif_files:
        with Image.open(tif_path) as img:
            rgb_img = img.convert('RGB')
            new_path = tif_path.replace('.tif', '.png')
            rgb_img.save(new_path)

        os.remove(tif_path)
    
    print(f"Conversión completada para la clase {class_name}.")

## *Verificación de disponibilidad de GPU*

Esta celda comprueba si TensorFlow detecta una GPU en el entorno actual. Es útil para confirmar que la aceleración por hardware está activa y que se aprovechará la GPU durante el entrenamiento del modelo.

In [ ]:
print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_gpu_available())

## Preproceso

In [ ]:
def plot_accuracy_loss(history):

    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

### Carga de los datos

In [ ]:
dataset_path = './dataset'
image_size = (224, 224)

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size = image_size,
    shuffle = True,
    seed = 23
)

class_names = dataset.class_names
print("Classes:", class_names)

### Visualización básica

In [ ]:
'''
for images, labels in dataset.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)

    plt.figure(figsize=(15, 15))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Label: {labels[i].numpy()} ({class_names[labels[i].numpy()]})")
        plt.axis("off")

### Partición del conjunto de datos

In [ ]:
X = []
y = []

for images, labels in dataset:
    X.append(images.numpy())
    y.append(labels.numpy())

X = np.concatenate(X)
y = np.concatenate(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23, stratify=y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=23, stratify=y_test)

print("Train:", X_train.shape, y_train.shape)
print("Validation:  ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)

In [ ]:
'''
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
df_train = pd.DataFrame(X_train_flattened)
df_train['target'] = y_train
df_train.head()

Reordenaremos las columnas de manera que la variable target sea la primera

In [ ]:
'''
cols = list(df_train.columns)
cols.remove('target')
cols.insert(0,'target')
df_train = df_train.reindex(columns=cols)
df_train.head()

In [ ]:
#df_train.iloc[:, :1000].describe().T

In [ ]:
#df_train['target'].value_counts()

In [ ]:
augmenter = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

augmentations_per_image = 3

X_augmented = []
y_augmented = []

for img, label in zip(X_train, y_train):
    img = np.expand_dims(img, 0)
    aug_iter = augmenter.flow(img, batch_size=1)
    for _ in range(augmentations_per_image):
        aug_img = next(aug_iter)[0].astype(np.uint8)
        X_augmented.append(aug_img)
        y_augmented.append(label)

X_augmented = np.array(X_augmented)
y_augmented = np.array(y_augmented)

X_train_augmented = np.concatenate([X_train, X_augmented])
y_train_augmented = np.concatenate([y_train, y_augmented])

print("Original training set size:", X_train.shape)
print("Augmented training set size:", X_train_augmented.shape)

In [ ]:
data_augmentation = Sequential([
    RandomFlip(),
    RandomRotation(0.1),
    RandomZoom(0.1),
    RandomTranslation(0.1, 0.1)
])

## Entrenar la CNN

In [ ]:
EPOCHS = 30

In [ ]:
callbacks = [ModelCheckpoint('best_model.h5', monitor = 'val_accuracy', save_best_only = True, verbose = 1),
             ReduceLROnPlateau(monitor = 'val_accuracy', patience = 4, factor = 0.1, verbose = 1, min_lr = 1e-6),
             EarlyStopping(monitor = 'val_accuracy', patience = 5, verbose = 1)]

### InceptionV3

In [ ]:
inceptionV3_base_model = InceptionV3(include_top = False)

#### Freeze and Feature Extraction

In [ ]:
inceptionV3_base_model.trainable = False

In [ ]:
inceptionV3_inputs = Input(shape = (224, 224, 3))

# inceptionV3_x = data_augmentation(inceptionV3_inputs)

inceptionV3_x = tf.keras.applications.inception_v3.preprocess_input(inceptionV3_inputs)

inceptionV3_x = inceptionV3_base_model(inceptionV3_x)

inceptionV3_x = GlobalAveragePooling2D()(inceptionV3_x)

inceptionV3_outputs = Dense(3, activation = 'softmax')(inceptionV3_x)

inceptionV3_model = Model(inceptionV3_inputs, inceptionV3_outputs)

inceptionV3_model.summary()

In [ ]:
len(inceptionV3_base_model.layers)

In [ ]:
'''
tf.keras.utils.plot_model(
    inceptionV3_model,
    to_file = 'InceptionV3.png',
    show_shapes = True,
    show_dtype = False,
    show_layer_names = False,
    show_layer_activations = True,
    expand_nested = True
)
'''

In [ ]:
inceptionV3_model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = Adam(learning_rate = 0.001),
              metrics = ['accuracy'])

In [ ]:
inceptionV3_history = inceptionV3_model.fit(X_train, y_train, epochs = EPOCHS, validation_data = (X_val, y_val))

In [ ]:
acc = inceptionV3_history.history['accuracy']
val_acc = inceptionV3_history.history['val_accuracy']

loss = inceptionV3_history.history['loss']
val_loss = inceptionV3_history.history['val_loss']

In [ ]:
plot_accuracy_loss(inceptionV3_history)

#### Unfreeze and Fine-Tuning

In [ ]:
inceptionV3_base_model.trainable = True
for layer in inceptionV3_base_model.layers[:-50]:
    layer.trainable = False

In [ ]:
inceptionV3_model.summary()

In [ ]:
inceptionV3_model.compile(loss = 'sparse_categorical_crossentropy',
                          optimizer = Adam(learning_rate = 0.0001),
                          metrics = ['accuracy'])

In [ ]:
inceptionV3_history_fine = inceptionV3_model.fit(X_train, y_train, epochs = EPOCHS * 2, initial_epoch = EPOCHS, validation_data = (X_val, y_val))

In [ ]:
acc += inceptionV3_history_fine.history['accuracy']
val_acc += inceptionV3_history_fine.history['val_accuracy']

loss += inceptionV3_history_fine.history['loss']
val_loss += inceptionV3_history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([EPOCHS-1,EPOCHS-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.grid(True)
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([EPOCHS-1,EPOCHS-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.grid(True)
plt.xlabel('epoch')
plt.show()

#### Evaluate Model

In [ ]:
inceptionV3_model.evaluate(X_test, y_test)

In [ ]:
y_pred_probs = inceptionV3_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
n_classes = len(class_names)

y_score = inceptionV3_model.predict(X_test)

y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    if np.sum(y_test_bin[:, i]) == 0:
        print(f"Skipping class {i} ({class_names[i]}): no positive samples in y_test.")
        continue
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
plt.figure(figsize=(8, 6))
for i, color in zip(roc_auc, colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(class_names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()

### DenseNet201

In [ ]:
denseNet201_base_model = DenseNet201(include_top = False)

#### Freeze and Feature Extraction

In [ ]:
denseNet201_base_model.trainable = False

In [ ]:
denseNet201_inputs = Input(shape = (224, 224, 3))

denseNet201_x = data_augmentation(denseNet201_inputs)

denseNet201_x = tf.keras.applications.densenet.preprocess_input(denseNet201_x)

denseNet201_x = denseNet201_base_model(denseNet201_x)

denseNet201_x = GlobalAveragePooling2D()(denseNet201_x)

denseNet201_outputs = Dense(3, activation = 'softmax')(denseNet201_x)

denseNet201_model = Model(denseNet201_inputs, denseNet201_outputs)

denseNet201_model.summary()

In [ ]:
len(denseNet201_base_model.layers)

In [ ]:
denseNet201_model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = Adam(learning_rate = 0.001),
              metrics = ['accuracy'])

In [ ]:
denseNet201_history = denseNet201_model.fit(X_train_augmented, y_train_augmented, epochs = EPOCHS, validation_data = (X_val, y_val))

In [ ]:
acc = denseNet201_history.history['accuracy']
val_acc = denseNet201_history.history['val_accuracy']

loss = denseNet201_history.history['loss']
val_loss = denseNet201_history.history['val_loss']

In [ ]:
plot_accuracy_loss(denseNet201_history)

#### Unfreeze and Fine-Tuning

In [ ]:
denseNet201_base_model.trainable = True
for layer in denseNet201_base_model.layers[:-200]:
    layer.trainable = False

In [ ]:
denseNet201_model.summary()

In [ ]:
denseNet201_model.compile(loss = 'sparse_categorical_crossentropy',
                          optimizer = Adam(learning_rate = 0.0001),
                          metrics = ['accuracy'])

In [ ]:
denseNet201_history_fine = denseNet201_model.fit(X_train_augmented, y_train_augmented, epochs = EPOCHS * 2, initial_epoch = EPOCHS, validation_data = (X_val, y_val))

In [ ]:
acc += denseNet201_history_fine.history['accuracy']
val_acc += denseNet201_history_fine.history['val_accuracy']

loss += denseNet201_history_fine.history['loss']
val_loss += denseNet201_history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([EPOCHS-1,EPOCHS-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.grid(True)
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([EPOCHS-1,EPOCHS-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.grid(True)
plt.xlabel('epoch')
plt.show()

#### Evaluate Model

In [ ]:
denseNet201_model.evaluate(X_test, y_test)

In [ ]:
y_pred_probs = denseNet201_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
n_classes = len(class_names)

y_score = denseNet201_model.predict(X_test)

y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    if np.sum(y_test_bin[:, i]) == 0:
        print(f"Skipping class {i} ({class_names[i]}): no positive samples in y_test.")
        continue
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
plt.figure(figsize=(8, 6))
for i, color in zip(roc_auc, colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(class_names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()

### MobileNetV2

In [ ]:
mobileNetV2_base_model = MobileNetV2(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
mobileNetV2_base_model.trainable = False

#### Freeze and Feature Extraction

In [ ]:
mobileNetV2_inputs = Input(shape = (224, 224, 3))

mobileNetV2_x = data_augmentation(mobileNetV2_inputs)

mobileNetV2_x = tf.keras.applications.mobilenet_v2.preprocess_input(mobileNetV2_x)

mobileNetV2_x = mobileNetV2_base_model(mobileNetV2_x)

mobileNetV2_x = GlobalAveragePooling2D()(mobileNetV2_x)

mobileNetV2_outputs = Dense(3, activation = 'softmax')(mobileNetV2_x)

mobileNetV2_model = Model(mobileNetV2_inputs, mobileNetV2_outputs)

mobileNetV2_model.summary()

In [ ]:
len(mobileNetV2_base_model.layers)

In [ ]:
'''
tf.keras.utils.plot_model(
    mobileNetV2_model,
    to_file = 'MobileNetV2.png',
    show_shapes = True,
    show_dtype = False,
    show_layer_names = False,
    show_layer_activations = True,
    expand_nested = True
)
'''

In [ ]:
mobileNetV2_model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = Adam(learning_rate = 0.001),
              metrics = ['accuracy'])

In [ ]:
mobileNetV2_history = mobileNetV2_model.fit(X_train_augmented, y_train_augmented, epochs = EPOCHS, validation_data = (X_val, y_val))

In [ ]:
acc = mobileNetV2_history.history['accuracy']
val_acc = mobileNetV2_history.history['val_accuracy']

loss = mobileNetV2_history.history['loss']
val_loss = mobileNetV2_history.history['val_loss']

In [ ]:
plot_accuracy_loss(mobileNetV2_history)

#### Unfreeze and Fine-Tuning

In [ ]:
mobileNetV2_base_model.trainable = True
for layer in mobileNetV2_base_model.layers[:-5]:
    layer.trainable = False

In [ ]:
mobileNetV2_model.summary()

In [ ]:
mobileNetV2_model.compile(loss = 'sparse_categorical_crossentropy',
                          optimizer = Adam(learning_rate = 0.0001),
                          metrics = ['accuracy'])

In [ ]:
mobileNetV2_history_fine = mobileNetV2_model.fit(X_train_augmented, y_train_augmented, epochs = EPOCHS * 2, initial_epoch = EPOCHS, validation_data = (X_val, y_val))

In [ ]:
acc += mobileNetV2_history_fine.history['accuracy']
val_acc += mobileNetV2_history_fine.history['val_accuracy']

loss += mobileNetV2_history_fine.history['loss']
val_loss += mobileNetV2_history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([EPOCHS-1,EPOCHS-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.grid(True)
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([EPOCHS-1,EPOCHS-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.grid(True)
plt.xlabel('epoch')
plt.show()

#### Evaluate Model

In [ ]:
mobileNetV2_model.evaluate(X_test, y_test)

In [ ]:
y_pred_probs = mobileNetV2_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
n_classes = len(class_names)

y_score = mobileNetV2_model.predict(X_test)

y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    if np.sum(y_test_bin[:, i]) == 0:
        print(f"Skipping class {i} ({class_names[i]}): no positive samples in y_test.")
        continue
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
plt.figure(figsize=(8, 6))
for i, color in zip(roc_auc, colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(class_names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()

### EfficientNetV2B0

In [ ]:
efficientNetV2B0_base_model = EfficientNetV2B0(include_top = False)

#### Freeze and Feature Extraction

In [ ]:
efficientNetV2B0_base_model.trainable = False

In [ ]:
efficientNetV2B0_inputs = Input(shape = (224, 224, 3))

efficientNetV2B0_x = data_augmentation(efficientNetV2B0_inputs)

efficientNetV2B0_x = tf.keras.applications.efficientnet_v2.preprocess_input(efficientNetV2B0_x)

efficientNetV2B0_x = efficientNetV2B0_base_model(efficientNetV2B0_x)

efficientNetV2B0_x = GlobalAveragePooling2D()(efficientNetV2B0_x)

efficientNetV2B0_outputs = Dense(3, activation = 'softmax')(efficientNetV2B0_x)

efficientNetV2B0_model = Model(efficientNetV2B0_inputs, efficientNetV2B0_outputs)

efficientNetV2B0_model.summary()

In [ ]:
len(efficientNetV2B0_base_model.layers)

In [ ]:
efficientNetV2B0_model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = Adam(learning_rate = 0.001),
              metrics = ['accuracy'])

In [ ]:
efficientNetV2B0_history = efficientNetV2B0_model.fit(X_train_augmented, y_train_augmented, epochs = EPOCHS, validation_data = (X_val, y_val))

In [ ]:
acc = efficientNetV2B0_history.history['accuracy']
val_acc = efficientNetV2B0_history.history['val_accuracy']

loss = efficientNetV2B0_history.history['loss']
val_loss = efficientNetV2B0_history.history['val_loss']

In [ ]:
plot_accuracy_loss(efficientNetV2B0_history)

#### Unfreeze and Fine-Tuning

In [ ]:
efficientNetV2B0_base_model.trainable = True
for layer in efficientNetV2B0_base_model.layers[:-50]:
    layer.trainable = False

In [ ]:
efficientNetV2B0_model.summary()

In [ ]:
efficientNetV2B0_model.compile(loss = 'sparse_categorical_crossentropy',
                          optimizer = Adam(learning_rate = 0.0001),
                          metrics = ['accuracy'])

In [ ]:
efficientNetV2B0_history_fine = efficientNetV2B0_model.fit(X_train_augmented, y_train_augmented, epochs = EPOCHS * 2, initial_epoch = EPOCHS, validation_data = (X_val, y_val))

In [ ]:
acc += efficientNetV2B0_history_fine.history['accuracy']
val_acc += efficientNetV2B0_history_fine.history['val_accuracy']

loss += efficientNetV2B0_history_fine.history['loss']
val_loss += efficientNetV2B0_history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([EPOCHS-1,EPOCHS-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.grid(True)
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([EPOCHS-1,EPOCHS-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.grid(True)
plt.xlabel('epoch')
plt.show()

#### Evaluate Model

In [ ]:
efficientNetV2B0_model.evaluate(X_test, y_test)

In [ ]:
y_pred_probs = efficientNetV2B0_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
n_classes = len(class_names)

y_score = efficientNetV2B0_model.predict(X_test)

y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    if np.sum(y_test_bin[:, i]) == 0:
        print(f"Skipping class {i} ({class_names[i]}): no positive samples in y_test.")
        continue
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
plt.figure(figsize=(8, 6))
for i, color in zip(roc_auc, colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(class_names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()

### ResNet50V2

In [ ]:
resNet50V2_base_model = ResNet50V2(include_top = False)

#### Freeze and Feature Extraction

In [ ]:
resNet50V2_base_model.trainable = False

In [ ]:
resNet50V2_inputs = Input(shape = (224, 224, 3))

resNet50V2_x = data_augmentation(resNet50V2_inputs)

resNet50V2_x = tf.keras.applications.resnet_v2.preprocess_input(resNet50V2_x)

resNet50V2_x = resNet50V2_base_model(resNet50V2_x)

resNet50V2_x = GlobalAveragePooling2D()(resNet50V2_x)

resNet50V2_outputs = Dense(3, activation = 'softmax')(resNet50V2_x)

resNet50V2_model = Model(resNet50V2_inputs, resNet50V2_outputs)

resNet50V2_model.summary()

In [ ]:
len(resNet50V2_base_model.layers)

In [ ]:
resNet50V2_model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = Adam(learning_rate = 0.001),
              metrics = ['accuracy'])

In [ ]:
resNet50V2_history = resNet50V2_model.fit(X_train_augmented, y_train_augmented, epochs = EPOCHS, validation_data = (X_val, y_val))

In [ ]:
acc = resNet50V2_history.history['accuracy']
val_acc = resNet50V2_history.history['val_accuracy']

loss = resNet50V2_history.history['loss']
val_loss = resNet50V2_history.history['val_loss']

In [ ]:
plot_accuracy_loss(resNet50V2_history)

#### Unfreeze and Fine-Tuning

In [ ]:
resNet50V2_base_model.trainable = True
for layer in resNet50V2_base_model.layers[:-20]:
    layer.trainable = False

In [ ]:
resNet50V2_model.summary()

In [ ]:
resNet50V2_model.compile(loss = 'sparse_categorical_crossentropy',
                          optimizer = Adam(learning_rate = 0.0001),
                          metrics = ['accuracy'])

In [ ]:
resNet50V2_history_fine = resNet50V2_model.fit(X_train_augmented, y_train_augmented, epochs = EPOCHS * 2, initial_epoch = EPOCHS, validation_data = (X_val, y_val))

In [ ]:
acc += resNet50V2_history_fine.history['accuracy']
val_acc += resNet50V2_history_fine.history['val_accuracy']

loss += resNet50V2_history_fine.history['loss']
val_loss += resNet50V2_history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([EPOCHS-1,EPOCHS-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.grid(True)
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([EPOCHS-1,EPOCHS-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.grid(True)
plt.xlabel('epoch')
plt.show()

#### Evaluate Model

In [ ]:
resNet50V2_model.evaluate(X_test, y_test)

In [ ]:
y_pred_probs = resNet50V2_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
n_classes = len(class_names)

y_score = resNet50V2_model.predict(X_test)

y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    if np.sum(y_test_bin[:, i]) == 0:
        print(f"Skipping class {i} ({class_names[i]}): no positive samples in y_test.")
        continue
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
plt.figure(figsize=(8, 6))
for i, color in zip(roc_auc, colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(class_names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()

### VGG16

In [ ]:
vgg16_base_model = VGG16(include_top = False)

In [ ]:
vgg16_base_model.trainable = False

In [ ]:
vgg16_inputs = Input(shape = (224, 224, 3))

vgg16_x = data_augmentation(vgg16_inputs)

vgg16_x = tf.keras.applications.vgg16.preprocess_input(vgg16_x)

vgg16_x = vgg16_base_model(vgg16_x)

vgg16_x = GlobalAveragePooling2D()(vgg16_x)

vgg16_outputs = Dense(3, activation = 'softmax')(vgg16_x)

vgg16_model = Model(vgg16_inputs, vgg16_outputs)

vgg16_model.summary()

In [ ]:
len(vgg16_base_model.layers)

In [ ]:
vgg16_model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = Adam(learning_rate = 0.001),
              metrics = ['accuracy'])

In [ ]:
vgg16_history = vgg16_model.fit(X_train_augmented, y_train_augmented, epochs = EPOCHS, validation_data = (X_val, y_val), batch_size=8)

In [ ]:
acc = vgg16_history.history['accuracy']
val_acc = vgg16_history.history['val_accuracy']

loss = vgg16_history.history['loss']
val_loss = vgg16_history.history['val_loss']

In [ ]:
plot_accuracy_loss(vgg16_history)

#### Unfreeze and Fine-Tuning

In [ ]:
vgg16_base_model.trainable = True
for layer in vgg16_base_model.layers[:-5]:
    layer.trainable = False

In [ ]:
vgg16_model.summary()

In [ ]:
vgg16_model.compile(loss = 'sparse_categorical_crossentropy',
                          optimizer = Adam(learning_rate = 0.0001),
                          metrics = ['accuracy'])

In [ ]:
vgg16_history_fine = vgg16_model.fit(X_train_augmented, y_train_augmented, epochs = EPOCHS * 2, initial_epoch = EPOCHS, validation_data = (X_val, y_val), batch_size=8)

In [ ]:
acc += vgg16_history_fine.history['accuracy']
val_acc += vgg16_history_fine.history['val_accuracy']

loss += vgg16_history_fine.history['loss']
val_loss += vgg16_history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([EPOCHS-1,EPOCHS-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.grid(True)
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([EPOCHS-1,EPOCHS-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.grid(True)
plt.xlabel('epoch')
plt.show()

#### Evaluate Model

In [ ]:
vgg16_model.evaluate(X_test, y_test, batch_size=8)

In [ ]:
y_pred_probs = vgg16_model.predict(X_test, batch_size=8)
y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
n_classes = len(class_names)

y_score = vgg16_model.predict(X_test, batch_size=8)

y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    if np.sum(y_test_bin[:, i]) == 0:
        print(f"Skipping class {i} ({class_names[i]}): no positive samples in y_test.")
        continue
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
plt.figure(figsize=(8, 6))
for i, color in zip(roc_auc, colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(class_names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC Curve')
plt.legend(loc="lower right")
plt.grid()
plt.show()